In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip install --upgrade google-cloud-storage
!pip install llama-index
!pip install llama-index-embeddings-huggingface llama_index datasets PyMuPDF huggingface_hub transformers llama-index-embeddings-instructor
! pip install sentence-transformers
! pip install llama-index-embeddings-langchain
! pip install langchain langchain-community langchain-core
!pip install python-dotenv
!pip install llama-index-readers-gcs

In [4]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
%cd drive/My Drive/Colab Notebooks/Stanford/CS224N/final-project/datasets

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# install gcsfuse
# !echo deb http://packages.cloud.google.com/apt gcsfuse-bionic main > /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
# !apt -qq update
# !apt -qq install gcsfuse

In [6]:
# verify that you are in the right directory
! ls #arxiv-papers

In [7]:
from google.colab import userdata
import os

os.environ['hf'] = userdata.get('hf')

In [ ]:
# !mkdir gcp_directory

In [2]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

In [ ]:
# !gcsfuse --implicit-dirs arxiv-dataset gcp_directory

In [ ]:
# !ls gcp_directory/arxiv/arxiv/pdf

In [ ]:
# testing small files
!python process-arxiv-data.py --main_directory "/content/drive/My Drive/Colab Notebooks/Stanford/CS224N/final-project/datasets/arxiv-papers" --model_name "WhereIsAI/UAE-Large-V1" --hf_username "amrachraf" --hf_dataset_name "arXiv-full-text-chunked-test" --chunk_size_gb 40 --local_path "/content/drive/MyDrive/Colab Notebooks/Stanford/CS224N/final-project/datasets/temp" --base_chunk_count 0

In [8]:
service_account_key_path = "amrr-382100-29e33e7c03a1.json"
bucket_name = "arxiv-dataset"
prefix = "arxiv/arxiv/pdf"

In [9]:
!python process-arxiv-data-v3.py --bucket_name $bucket_name --prefix $prefix --folders 2403 2402 2401 2312 2311 2310 2309 2308 2307 \
2306 2305 2304 2303 2302 2301 2212 2211 2210 2209  \
 --service_account_key_path $service_account_key_path \
 --model_name "WhereIsAI/UAE-Large-V1" --hf_username "amrachraf" \
 --hf_dataset_name "arXiv-full-text-chunked" --chunk_size_gb 0.1 \
 --local_path "/content/drive/MyDrive/Colab Notebooks/Stanford/CS224N/final-project/datasets/temp" \
 --base_chunk_count 4 --num_files_limit 200 --max_files 400 --use_folder_limit True

In [ ]:
# testing multithreading
!python process-arxiv-data-v2.py --bucket_name $bucket_name --prefix $prefix --folders 2403 2402 2401 2312 2311 2310 2309 2308 2307 \
2306 2305 2304 2303 2302 2301 2212 2211 2210 2209  \
 --service_account_key_path $service_account_key_path \
 --model_name "WhereIsAI/UAE-Large-V1" --hf_username "amrachraf" \
 --hf_dataset_name "arXiv-full-text-chunked-test2" --chunk_size_gb 0.1 \
 --local_path "/content/drive/MyDrive/Colab Notebooks/Stanford/CS224N/final-project/datasets/temp" \
 --base_chunk_count 0 --num_files_limit 2 --max_files 10 --use_folder_limit True

In [ ]:
# testing multiprocessing
!python process-arxiv-data-v3.py --bucket_name $bucket_name --prefix $prefix --folders 2403 2402 2401 2312 2311 2310 2309 2308 2307 \
2306 2305 2304 2303 2302 2301 2212 2211 2210 2209  \
 --service_account_key_path $service_account_key_path \
 --model_name "WhereIsAI/UAE-Large-V1" --hf_username "amrachraf" \
 --hf_dataset_name "arXiv-full-text-chunked-test" --chunk_size_gb 0.1 \
 --local_path "/content/drive/MyDrive/Colab Notebooks/Stanford/CS224N/final-project/datasets/temp" \
 --base_chunk_count 0 --num_files_limit 2 --max_files 10 --use_folder_limit True

In [ ]:
from google.cloud import storage

In [ ]:
ls

In [ ]:
# service_account_key_path = "amrr-382100-29e33e7c03a1.json"
# bucket_name = "arxiv-dataset"
# prefix = "arxiv/arxiv/pdf"

# def list_files_in_bucket(bucket_name, prefix, service_account_key_path):
#     client = storage.Client.from_service_account_json(service_account_key_path)
#     bucket = client.bucket(bucket_name)
#     blobs = bucket.list_blobs(prefix=prefix)
#     for page in blobs.pages:
#         yield from [blob.name for blob in page]

In [ ]:
# a = list_files_in_bucket(bucket_name, prefix, service_account_key_path)

In [ ]:
# for i, file in enumerate(list_files_in_bucket(bucket_name, prefix, service_account_key_path)):
#   print(i)

In [ ]:
from huggingface_hub import notebook_login, login
from google.colab import userdata

hfToken = userdata.get('hf')
# notebook_login()
login(hfToken)

In [ ]:
import sys
from numbers import Number
from collections import deque
from collections.abc import Set, Mapping


ZERO_DEPTH_BASES = (str, bytes, Number, range, bytearray)


def getsize(obj_0):
    """Recursively iterate to sum size of object & members."""
    _seen_ids = set()
    def inner(obj):
        obj_id = id(obj)
        if obj_id in _seen_ids:
            return 0
        _seen_ids.add(obj_id)
        size = sys.getsizeof(obj)
        if isinstance(obj, ZERO_DEPTH_BASES):
            pass # bypass remaining control flow and return
        elif isinstance(obj, (tuple, list, Set, deque)):
            size += sum(inner(i) for i in obj)
        elif isinstance(obj, Mapping) or hasattr(obj, 'items'):
            size += sum(inner(k) + inner(v) for k, v in getattr(obj, 'items')())
        # Check for custom object instances - may subclass above too
        if hasattr(obj, '__dict__'):
            size += inner(vars(obj))
        if hasattr(obj, '__slots__'): # can have __slots__ with __dict__
            size += sum(inner(getattr(obj, s)) for s in obj.__slots__ if hasattr(obj, s))
        return size
    return inner(obj_0)

In [ ]:
getsize(documents)/1e+9